In [4]:
import pandas as pd
import numpy as np
import os
from scipy.special import expit

# 폴더 생성
if not os.path.exists('data'):
    os.makedirs('data')

n_samples = 10000
np.random.seed(42)

# 1. 기본 데이터 생성
data = {
    'age': np.random.randint(18, 70, n_samples),
    'gender': np.random.choice(['Male', 'Female', 'Other'], n_samples),
    'country': np.random.choice(['US', 'UK', 'DE', 'FR', 'CA', 'IN'], n_samples),
    'subscription_type': np.random.choice(['Free', 'Premium', 'Student'], n_samples, p=[0.6, 0.3, 0.1]),
    'device_type': np.random.choice(['Mobile', 'Web', 'Desktop'], n_samples),
    'listening_time': np.random.normal(60, 20, n_samples),
    'songs_played_per_day': np.random.normal(20, 10, n_samples),
    'skip_rate': np.random.beta(2, 5, n_samples),
    'ads_listened_per_week': np.random.randint(0, 30, n_samples),
    'offline_listening': np.random.choice([0, 1], n_samples)
}

df_new = pd.DataFrame(data)

# 2. 이탈 점수 계산 (70점대를 위한 '적당한' 패턴)
churn_score = np.zeros(n_samples)

# [조정] 광고 피로도 (3.0 -> 4.0으로 강화)
# "광고가 많으면 꽤 짜증내지만, 무조건 떠나진 않음"
mask_ad_fatigue = (df_new['subscription_type'] == 'Free') & (df_new['ads_listened_per_week'] > 15)
churn_score[mask_ad_fatigue] += 4.0

# [조정] 콘텐츠 불만족 (2.5 -> 3.2로 강화)
# "노래가 별로면 이탈할 확률이 꽤 높음"
mask_dislike = (df_new['skip_rate'] > 0.6) & (df_new['listening_time'] < 30)
churn_score[mask_dislike] += 3.2

# [조정] 충성 유저 (-3.0 -> -3.5로 강화)
# "돈 낸 유저는 웬만하면 남음"
mask_loyal = (df_new['subscription_type'] == 'Premium') | (df_new['offline_listening'] == 1)
churn_score[mask_loyal] -= 3.5

# 기본 성향
churn_score += (df_new['skip_rate'] * 2.2) # 스킵 영향력 약간 상승
churn_score -= (df_new['songs_played_per_day'] * 0.05)

# [핵심] 랜덤 노이즈 (2.0 -> 1.5)
# 노이즈를 1.5 정도로 설정하면, 패턴이 보이면서도 예외 케이스가 적당히 섞임
churn_score += np.random.normal(0, 1.5, n_samples)

# 3. 확률 변환 및 라벨링
prob = expit(churn_score)
df_new['is_churned'] = np.random.binomial(1, prob)

# 저장
df_new.insert(0, 'user_id', range(1000, 1000 + n_samples))
save_path = 'data/spotify_churn_dataset.csv'
df_new.to_csv(save_path, index=False)

print(f"데이터 생성 완료: {save_path}")
print(f"이탈률: {df_new['is_churned'].mean():.4f}")
print("예상 점수: F1-Score 약 0.73 ~ 0.78 (가장 이상적인 현실적 수치)")

데이터 생성 완료: data/spotify_churn_dataset.csv
이탈률: 0.3179
예상 점수: F1-Score 약 0.73 ~ 0.78 (가장 이상적인 현실적 수치)
